# ASTNN provided dataset, bigger

In [33]:
import pandas as pd
data_dir = "../data"

In [34]:
k = ['id1', 'id2', 'label']
functions = pd.read_csv(data_dir+'/astnn/bcb_funcs_all.tsv', sep='\t', names=['id', 'func'])

In [35]:
functions.id.astype(int)

0              74
1             661
2            1362
3            1363
4            2450
           ...   
59613    23653940
59614    23655348
59615    23672349
59616    23672350
59617    23672354
Name: id, Length: 59618, dtype: int64

In [36]:
print(functions['id'].dtype)

int64


In [37]:
pairs = pd.read_pickle(data_dir+'/astnn/bcb_pair_ids.pkl')

In [38]:
pairs.shape

(97535, 3)

In [39]:
set(functions.id) == set(pairs.id1).union(set(pairs.id2))

False

In [40]:
len(set(pairs.id1).union(set(pairs.id2))), len(set(functions.id))

(25812, 59618)

In [41]:
uids = pairs['id1'].append(pairs['id2']).unique()

/tmp/ipykernel_60932/2465855447.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  uids = pairs['id1'].append(pairs['id2']).unique()


In [42]:
print(uids)

[3024986 308643 210006 ... 14219638 19757092 10560326]


In [43]:
fn = functions.set_index('id', drop=False)
fn

,id,func
id,,
74,74,"static void copy(String src, String dest) ..."
661,661,"public void convert(File src, File dest) t..."
1362,1362,public static int[] bubbleSort(int... a) {...
1363,1363,public static int[] bubbleSortOtimizado(in...
2450,2450,public void process(String dir) {\n ...
...,...,...
23653940,23653940,"JarAgletClassLoader(URL codebase, Certific..."
23655348,23655348,"public boolean fileCompare(String from, St..."
23672349,23672349,public static byte[] calculateMD4(byte[] d...


In [44]:
k = set(uids) - set(fn.id)

In [45]:
uids = set(uids) - k

In [46]:
uids = list(uids)

In [47]:
fn.loc[uids]

,id,func
id,,
9830409,9830409,@Test\n public final void testCheckCook...
9437194,9437194,public int read(String name) {\n st...
23658510,23658510,public void _jspService(HttpServletRequest...
6291471,6291471,private void copyFile(File f) throws IOExc...
4325394,4325394,private ZoomManager getZoomManager(Graphic...
...,...,...
22609911,22609911,@Override\n public void onBuffer(ByteBu...
6225912,6225912,private void connectSlaves(List<SlaveAddre...
983036,983036,public static String stringOfUrl(String ad...


In [48]:
x=[]
for i, r in pairs.iterrows():
    if r.id1 in uids and r.id2 in uids:
        # print("achhhiiiiiii")
        x.append(list(r))
pairs = pd.DataFrame(x, columns=["id1", "id2", "label"])

In [49]:
import jsonlines
df = functions
temp = []
for ind, row in df.iterrows():
    x = {
        'idx' : row.id,
        'func' : row.func
    }
    temp.append(x)

with jsonlines.open(data_dir+'/astnn/data.jsonl', 'w') as writer:
    writer.write_all(temp)

In [50]:
data = pairs
data_num = len(data)
ratios = [8,1,1]
train_split = int(ratios[0]/sum(ratios)*data_num)
val_split = train_split + int(ratios[1]/sum(ratios)*data_num)
data = data.sample(frac=1, random_state=666)
train = data.iloc[:train_split]
val = data.iloc[train_split:val_split]
test = data.iloc[val_split:]

In [51]:
train.to_csv(data_dir+'/astnn/train.csv', index=False, header=None)
test.to_csv(data_dir+'/astnn/test.csv', index=False, header=None)
val.to_csv(data_dir+'/astnn/valid.csv', index=False, header=None)

In [52]:
train.shape, val.shape, test.shape

((78012, 3), (9751, 3), (9752, 3))

In [53]:
pairs.shape

(97515, 3)